In [2]:
import pandas as pd
import numpy as np
import cv2 
import imutils
import time
import copy
from models.TrackNet.tracknet import trackNet

# Ball Tracking: Applying TrackNet Pre-Trained Model
* Input: mp4 video
* Ouptput: Coordinates, actions, to JSON file
* Model: TrackNet

In [8]:
#input path
input_video_path = 'VideoInput/video_input1.mp4'

#Note: Will need to figure out how to replace with live feed from camera

In [16]:
# get video fps
video = cv2.VideoCapture(input_video_path)
fps = int(video.get(cv2.CAP_PROP_FPS))
print('fps: {}'.format(fps))

# video size
orig_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
orig_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
orig_length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print('dimensions:', orig_width, 'x', orig_height)
print('length:', orig_length)

fps: 30
dimensions: 1920 x 1080
length: 206
total frames: 30.0


In [18]:
#split input video into frames
frames = []
#get initial frame
success, frame = video.read()
while success: 
    frame = cv2.resize(frame, (orig_width, orig_height))
    frames.append(frame)
   
    #iterate to next frame
    success, frame = video.read()
video.release()

In [26]:
#model input parameters
n_classes = 256
width, height = 640, 360
verbose = True
save_weights_path = 'WeightsTracknet/model.1'

#load in TrackNet model
modelFN = trackNet
m = modelFN(n_classes, input_height=height, input_width=width)
m.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
m.load_weights(save_weights_path)

In [36]:
#frame counter
frame_ctr = 0

pre_coords = []

for frame in frames:
    if (verbose) and (frame_ctr % 10 == 0):
        print('Frame:', frame_ctr)

    #convert to TrackNet width x height & float
    img = cv2.resize(frame, (width, height)).astype(np.float32)

    #ordering of TrackNet is 'channels_first'
    X = np.rollaxis(img, 2, 0)
    
    # predict heatmap
    pr = m.predict(np.array([X]))[0]

    # TrackNet output is ( net_output_height*model_output_width , n_classes )
    # reshape image as ( net_output_height, model_output_width , n_classes(depth) )
    # cv2 image must be numpy.uint8, convert numpy.int64 to numpy.uint8
    pr = pr.reshape((height, width, n_classes)).argmax(axis=2).astype(np.uint8)  

    # reshape the image size as original input image
    heatmap = cv2.resize(pr, (orig_width, orig_height))

    # heatmap is converted into a binary image by threshold method.
    ret, heatmap = cv2.threshold(heatmap, 127, 255, cv2.THRESH_BINARY)
    
    # find the circle in image with 2<=radius<=7
    circles = cv2.HoughCircles(heatmap, cv2.HOUGH_GRADIENT, dp=1, minDist=1, param1=50, param2=2, minRadius=2,
                              maxRadius=7)
    
    # check if there have any tennis be detected
    if circles is not None:
        # if only one tennis be detected
        if len(circles) == 1:
            x = int(circles[0][0][0])
            y = int(circles[0][0][1])
            pre_coords.append([x,y])
        else:
            pre_coords.append(None)
    else:
        pre_coords.append(None)

    frame_ctr += 1

Frame: 0


InvalidArgumentError: Graph execution error:

Detected at node 'model_7/max_pooling2d_9/MaxPool' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1776.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.1776.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\aaliy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\aaliy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "C:\Users\aaliy\AppData\

In [33]:
pred_coord

NameError: name 'pred_coord' is not defined